
<br>
==============================================================<br>
Restricted Boltzmann Machine features for digit classification<br>
==============================================================<br>
For greyscale image data where pixel values can be interpreted as degrees of<br>
blackness on a white background, like handwritten digit recognition, the<br>
Bernoulli Restricted Boltzmann machine model (:class:`BernoulliRBM<br>
<sklearn.neural_network.BernoulliRBM>`) can perform effective non-linear<br>
feature extraction.<br>
In order to learn good latent representations from a small dataset, we<br>
artificially generate more labeled data by perturbing the training data with<br>
linear shifts of 1 pixel in each direction.<br>
This example shows how to build a classification pipeline with a BernoulliRBM<br>
feature extractor and a :class:`LogisticRegression<br>
<sklearn.linear_model.LogisticRegression>` classifier. The hyperparameters<br>
of the entire model (learning rate, hidden layer size, regularization)<br>
were optimized by grid search, but the search is not reproduced here because<br>
of runtime constraints.<br>
Logistic regression on raw pixel values is presented for comparison. The<br>
example shows that the features extracted by the BernoulliRBM help improve the<br>
classification accuracy.<br>


In [ ]:
print(__doc__)

Authors: Yann N. Dauphin, Vlad Niculae, Gabriel Synnaeve<br>
License: BSD

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from scipy.ndimage import convolve
from sklearn import linear_model, datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.base import clone

#############################################################################<br>
Setting up

In [ ]:
def nudge_dataset(X, Y):
    """
    This produces a dataset 5 times bigger than the original one,
    by moving the 8x8 images in X around by 1px to left, right, down, up
    """
    direction_vectors = [
        [[0, 1, 0],
         [0, 0, 0],
         [0, 0, 0]],
        [[0, 0, 0],
         [1, 0, 0],
         [0, 0, 0]],
        [[0, 0, 0],
         [0, 0, 1],
         [0, 0, 0]],
        [[0, 0, 0],
         [0, 0, 0],
         [0, 1, 0]]]
    def shift(x, w):
        return convolve(x.reshape((8, 8)), mode='constant', weights=w).ravel()
    X = np.concatenate([X] +
                       [np.apply_along_axis(shift, 1, X, vector)
                        for vector in direction_vectors])
    Y = np.concatenate([Y for _ in range(5)], axis=0)
    return X, Y

Load Data

In [ ]:
X, y = datasets.load_digits(return_X_y=True)
X = np.asarray(X, 'float32')
X, Y = nudge_dataset(X, y)
X = (X - np.min(X, 0)) / (np.max(X, 0) + 0.0001)  # 0-1 scaling

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=0)

Models we will use

In [ ]:
logistic = linear_model.LogisticRegression(solver='newton-cg', tol=1)
rbm = BernoulliRBM(random_state=0, verbose=True)

In [ ]:
rbm_features_classifier = Pipeline(
    steps=[('rbm', rbm), ('logistic', logistic)])

#############################################################################<br>
Training

Hyper-parameters. These were set by cross-validation,<br>
using a GridSearchCV. Here we are not performing cross-validation to<br>
save time.

In [ ]:
rbm.learning_rate = 0.06
rbm.n_iter = 10
# More components tend to give better prediction performance, but larger
# fitting time
rbm.n_components = 100
logistic.C = 6000

Training RBM-Logistic Pipeline

In [ ]:
rbm_features_classifier.fit(X_train, Y_train)

Training the Logistic regression classifier directly on the pixel

In [ ]:
raw_pixel_classifier = clone(logistic)
raw_pixel_classifier.C = 100.
raw_pixel_classifier.fit(X_train, Y_train)

#############################################################################<br>
Evaluation

In [ ]:
Y_pred = rbm_features_classifier.predict(X_test)
print("Logistic regression using RBM features:\n%s\n" % (
    metrics.classification_report(Y_test, Y_pred)))

In [ ]:
Y_pred = raw_pixel_classifier.predict(X_test)
print("Logistic regression using raw pixel features:\n%s\n" % (
    metrics.classification_report(Y_test, Y_pred)))

#############################################################################<br>
Plotting

In [ ]:
plt.figure(figsize=(4.2, 4))
for i, comp in enumerate(rbm.components_):
    plt.subplot(10, 10, i + 1)
    plt.imshow(comp.reshape((8, 8)), cmap=plt.cm.gray_r,
               interpolation='nearest')
    plt.xticks(())
    plt.yticks(())
plt.suptitle('100 components extracted by RBM', fontsize=16)
plt.subplots_adjust(0.08, 0.02, 0.92, 0.85, 0.08, 0.23)

In [ ]:
plt.show()